In [1]:
import cv2

In [2]:
import mediapipe as mp

In [3]:
import cv2
import mediapipe as mp
import numpy as np
from ultralytics import YOLO

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

yolo_model = YOLO("yolov8n.pt", verbose=False)

def calculate_distance(point1, point2):
    return np.linalg.norm(point1 - point2)

def align_face(face_image, landmarks):
    left_eye = np.array([landmarks.landmark[468].x * face_image.shape[1], landmarks.landmark[468].y * face_image.shape[0]])
    right_eye = np.array([landmarks.landmark[473].x * face_image.shape[1], landmarks.landmark[473].y * face_image.shape[0]])

    dY = right_eye[1] - left_eye[1]
    dX = right_eye[0] - left_eye[0]
    angle = np.degrees(np.arctan2(dY, dX))

    center = (face_image.shape[1] // 2, face_image.shape[0] // 2)
    rot_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    aligned_face = cv2.warpAffine(face_image, rot_matrix, (face_image.shape[1], face_image.shape[0]), flags=cv2.INTER_LINEAR)

    return aligned_face

def save_results(image, measurements):
    with open("face_measurements.txt", "w") as f:
        for key, value in measurements.items():
            f.write(f"{key}: {value:.2f} mm\n")
    print("Results saved: captured_face.jpg and face_measurements.txt")

reference_object_width_pixels = 100
reference_object_width_mm = 25
pixel_to_mm_ratio = reference_object_width_mm / reference_object_width_pixels

cap = cv2.VideoCapture(0)
face_captured = False

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = yolo_model(rgb_frame)

    for result in results:
        boxes = result.boxes.xyxy
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)

            face_image = rgb_frame[y1:y2, x1:x2]
            face_results = face_mesh.process(face_image)

            if face_results.multi_face_landmarks:
                for face_landmarks in face_results.multi_face_landmarks:
                    aligned_face = align_face(face_image, face_landmarks)

                    left_eye = np.array([face_landmarks.landmark[468].x * (x2 - x1), face_landmarks.landmark[468].y * (y2 - y1)])
                    right_eye = np.array([face_landmarks.landmark[473].x * (x2 - x1), face_landmarks.landmark[473].y * (y2 - y1)])
                    nose_tip = np.array([face_landmarks.landmark[4].x * (x2 - x1), face_landmarks.landmark[4].y * (y2 - y1)])
                    chin = np.array([face_landmarks.landmark[152].x * (x2 - x1), face_landmarks.landmark[152].y * (y2 - y1)])

                    eye_distance = calculate_distance(left_eye, right_eye) * pixel_to_mm_ratio
                    nose_width = calculate_distance(nose_tip - np.array([20, 0]), nose_tip + np.array([20, 0])) * pixel_to_mm_ratio
                    face_length = calculate_distance(nose_tip, chin) * pixel_to_mm_ratio

                    mp_drawing = mp.solutions.drawing_utils
                    mp_drawing.draw_landmarks(
                        image=aligned_face,
                        landmark_list=face_landmarks,
                        connections=mp_face_mesh.FACEMESH_TESSELATION,
                        landmark_drawing_spec=None,
                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1)
                    )

                    cv2.putText(aligned_face, f"Eye Distance: {eye_distance:.2f} mm", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.putText(aligned_face, f"Nose Width: {nose_width:.2f} mm", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    cv2.putText(aligned_face, f"Face Length: {face_length:.2f} mm", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                    output_frame = cv2.cvtColor(aligned_face, cv2.COLOR_RGB2BGR)
                    cv2.imshow("Face Measurement System", output_frame)

                    if cv2.waitKey(1) & 0xFF == ord('s') and not face_captured:
                        measurements = {
                            "Eye Distance": eye_distance,
                            "Nose Width": nose_width,
                            "Face Length": face_length
                        }
                        save_results(aligned_face, measurements)
                        face_captured = True

    if cv2.waitKey(1) != -1: 
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 183.0ms
Speed: 8.4ms preprocess, 183.0ms inference, 3.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 175.2ms
Speed: 4.3ms preprocess, 175.2ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 135.2ms
Speed: 3.1ms preprocess, 135.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 147.6ms
Speed: 4.7ms preprocess, 147.6ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 150.0ms
Speed: 2.7ms preprocess, 150.0ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 143.3ms
Speed: 4.3ms preprocess, 143.3ms inference, 1.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 150.6ms
Speed: 3.9ms preprocess, 150.6ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 157.7ms
Speed: 4.5ms preprocess, 157.7ms inference, 2.0ms postprocess per image at